<a href="https://colab.research.google.com/github/madhumithasuresh-06/MachineLearningProjects/blob/main/FAKE_NEWS_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FAKE NEWS PREDICTION(TEXTUAL DATA) ---> USING LOGISTIC REGRESSION




IMPORTING DEPENDANCIES

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download('stopwords')
print(stopwords.words('english'))


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



DATA COLLECTION AND PREPROCESSING


In [ ]:
news_data=pd.read_csv('/content/news_own.csv')

news_data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",FAKE
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",FAKE
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",FAKE
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",FAKE
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",FAKE


In [ ]:
news_data.tail()

,title,text,subject,date,label
1992,Twitter bans ads from two Russian media outlet...,SAN FRANCISCO (Reuters) - Twitter Inc on Thurs...,politicsNews,"October 26, 2017",TRUE
1993,House panels seek documents on Puerto Rico uti...,WASHINGTON (Reuters) - A U.S. congressional co...,politicsNews,"October 26, 2017",TRUE
1994,"Trump releases some JFK files, blocks others u...",WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"October 26, 2017",TRUE
1995,Tillerson tells Myanmar army chief U.S. concer...,WASHINGTON (Reuters) - U.S. Secretary of State...,politicsNews,"October 27, 2017",TRUE
1996,"Exclusive: While advising Trump in 2016, ex-CI...",WASHINGTON (Reuters) - Former CIA director Jam...,politicsNews,"October 26, 2017",TRUE


In [ ]:
news_data.shape

(6335, 4)

HANDLING MISSING VALUE

In [ ]:
news_data.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


LABEL ENCODING

In [ ]:
news_data['label'].value_counts()

,count
label,
REAL,3171
FAKE,3164


In [ ]:
label_encode=LabelEncoder()
labels=label_encode.fit_transform(news_data['label'])
news_data['label']=labels
news_data['label'].value_counts()

,count
label,
1,999
0,998


#  1->real  | 0->fake

In [ ]:
news_data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


merging title and text

In [ ]:
news_data['content']=news_data['subject']+'  '+news_data['title']
news_data.head()

,title,text,subject,date,label,content
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,"politicsNews As U.S. budget fight looms, Repu..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,politicsNews U.S. military to accept transgen...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,politicsNews Senior U.S. Republican senator: ...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,politicsNews FBI Russia probe helped by Austr...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,politicsNews Trump wants Postal Service to ch...


HANDLING IMBALANCED DATASET

Prediction will be done based on the title to predict the label

stemming -> reducing words to its root

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_title=stemmed_content.split()
  stemmed_content=[port_stem.stem(words) for words in stemmed_content if not words in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_data['content']=news_data['content'].apply(stemming)
print(news_data['content'])

0       p l c n e w       u       b u g e   f g h   l ...
1       p l c n e w     u       l r     c c e p   r n ...
2       p l c n e w     e n r   u       r e p u b l c ...
3       p l c n e w     f b   r u   p r b e   h e l p ...
4       p l c n e w     r u p   w n   p l   e r v c e ...
                              ...                        
1992    p l c n e w     w e r   b n     f r   w   r u ...
1993    p l c n e w     h u e   p n e l   e e k   c u ...
1994    p l c n e w     r u p   r e l e e   e   j f k ...
1995    p l c n e w     l l e r n   e l l   n r   r   ...
1996    p l c n e w     e x c l u v e     w h l e   v ...
Name: content, Length: 1997, dtype: object


seperating data and lebel

In [ ]:
x=news_data['content'].values
y=news_data['label'].values

In [ ]:
print(x)
print(y)

['p     l     c     n     e     w     u     b     u     g     e     f     g     h     l     r     e     p     u     b     l     c     n     f     l     p     h     e     r     f     c     l     c     r     p'
 'p     l     c     n     e     w     u     l     r     c     c     e     p     r     n     g     e     n     e     r     r     e     c     r     u     n     n     p     e     n     g     n'
 'p     l     c     n     e     w     e     n     r     u     r     e     p     u     b     l     c     n     e     n     r     l     e     r     u     e     l     l     e     r     h     j     b'
 ...
 'p     l     c     n     e     w     r     u     p     r     e     l     e     e     e     j     f     k     f     l     e     b     l     c     k     h     e     r     u     n     e     r     p     r     e     u     r     e'
 'p     l     c     n     e     w     l     l     e     r     n     e     l     l     n     r     r     c     h     e     f     u     c     n     c     e     r     n     e

FEATURE EXTRACTION --> Converting text into respective numerical value

In [ ]:
vectorizer=TfidfVectorizer()

x=vectorizer.fit_transform(x)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
print(x)

['p     l     c     n     e     w     u     b     u     g     e     f     g     h     l     r     e     p     u     b     l     c     n     f     l     p     h     e     r     f     c     l     c     r     p'
 'p     l     c     n     e     w     u     l     r     c     c     e     p     r     n     g     e     n     e     r     r     e     c     r     u     n     n     p     e     n     g     n'
 'p     l     c     n     e     w     e     n     r     u     r     e     p     u     b     l     c     n     e     n     r     l     e     r     u     e     l     l     e     r     h     j     b'
 ...
 'p     l     c     n     e     w     r     u     p     r     e     l     e     e     e     j     f     k     f     l     e     b     l     c     k     h     e     r     u     n     e     r     p     r     e     u     r     e'
 'p     l     c     n     e     w     l     l     e     r     n     e     l     l     n     r     r     c     h     e     f     u     c     n     c     e     r     n     e

Train Test Split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
print(x.shape,x_train.shape,x_test.shape)

(6335, 6213) (5068, 6213) (1267, 6213)


Train the model

In [ ]:
model = LogisticRegression(class_weight='balanced',C=0.5)
model.fit(x_train, y_train)


LogisticRegression(C=0.5, class_weight='balanced')

cross validation

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print(scores.mean())


0.5237569060773481


Model Evaluation

In [ ]:
trainning_prediction=model.predict(x_train)
trainning_accuracy=accuracy_score(trainning_prediction,y_train)
print("Accuracy on trainning data:",trainning_accuracy)

Accuracy on trainning data: 0.999605367008682


In [ ]:
testing_prediction=model.predict(x_test)
testing_accuracy=accuracy_score(testing_prediction,y_test)
print("Accuracy on testing data:",testing_accuracy)

Accuracy on testing data: 0.5169692186266772


In [ ]:
classifier=svm.SVC(kernel='linear')
classifier.fit(x_train, y_train)

SVC(kernel='linear')

In [ ]:
trainning_prediction2=classifier.predict(x_train)
trainning_accuracy2=accuracy_score(trainning_prediction2,y_train)
print("Accuracy on trainning data:",trainning_accuracy2)

Accuracy on trainning data: 0.999605367008682


In [ ]:
testing_prediction2=classifier.predict(x_test)
testing_accuracy2=accuracy_score(testing_prediction2,y_test)
print("Accuracy on testing data:",testing_accuracy2)

Accuracy on testing data: 0.5169692186266772
